# <center>在多元文化的样本中情境因素塑造了东方、南方和西方国家电车困境中的道德判断实验数据分析的复刻  
</center>  
 
## <center>时天轲，叶子芸，欧星宇</center> 

## 原文献概述  
* 道德困境可以被描述为两个主要相互冲突的道德原则之间的抉择：功利主义和道义论。功利主义哲学认为，如果一项行动使最多人的福祉最大化，那么它在道德上是可以接受的；道义论哲学根据行为的内在特质（个人的权利和义务）来评估行为的道德性。双重效应学说指对于人们来说，伤害在作为好结果的无目的副作用时是可以接受的。然而，Greene等人认为，功利主义反应率的差异不能简单地用双重效应学说来解释。他们提出伤害意图（即伤害作为手段或副作用，指的是双重效应学说）和个人力量（即行为者是否必须使用个人努力杀死受害者并拯救更多人）在道德评级上相互作用的证据。  
* 原文献的研究检验了三个跨文化假设：1、个人力量对道德判断的影响在文化上是普遍的；2、个人力量和意图对道德判断的相互作用效应在文化上是普遍的；3、集体主义-个人主义对个人力量和意图影响道德判断的程度具有调节作用，其影响在更集体主义的文化中更强。  
* 在原文献中，实验分为三个部分：第一部分复刻了Greene等人的研究1，测试了个人力量在道德判断中作用的普遍性；第二部分复刻了Greene等人的研究2，测试了个人力量和意图对道德困境判断交互作用效应的普遍性；第三部分检验了集体主义缓和了意图和个人力量影响的假设。  


## 实验数据分析的复刻

### Workflow

![Image Name](https://cdn.kesci.com/upload/image/rkz1ehen1l.png?imageView2/0/w/960/h/960)

### 导入并清洗数据

In [16]:
# 导入 pymc 模型包，和 arviz 等分析工具
import pymc as pm
import arviz as az
import seaborn as sns
import scipy.stats as st
import numpy as np
import matplotlib.pyplot as plt
import xarray as xr
import pandas as pd
import ipywidgets
from scipy.stats import norm
from scipy.stats import gaussian_kde

# 忽略不必要的警告
import warnings
warnings.filterwarnings("ignore")


In [17]:
# 通过 pd.read_csv 加载数据 trolley_preprocessed.csv
df_raw = pd.read_csv('/home/mw/project/trolley_preprocessed.csv')

# 根据排除条件筛选数据
df = df_raw[(df_raw['careless_1'] == 2) & (df_raw['careless_2']
                                           == 2) & (df_raw['careless_3'] == 1)]  # 排除粗心大意被试的数据
df = df[df_raw['confusion'].isin([1, 2])]  # 排除完全无法理解材料被试的数据
df = df[df_raw['familiarity'].isin([1, 2, 3])]  # 排除熟悉道德两难问题被试的数据
df = df[df['technical_problems'] == 1]  # 排除发生技术故障被试的数据
df = df[df['native_language'] == 1]  # 排除不使用母语被试的数据


In [18]:
# 分别提取需要研究的研究1a，研究1b，研究2a，研究2b的数据
# 研究1a第一个材料（footbridge pole）
df_1a1 = df[["trolley_1_rate", "trolley_attention","survey_name"]]  # 选取与材料有关的变量
df_1a1.dropna(axis=0, how='any', inplace=True)  # 删除所有存在缺失值的行
df_1a1 = df_1a1[(df_1a1['trolley_attention'] == 2)]  # 排除注意力测试失败被试的数据
df_1a1 = df_1a1[["trolley_1_rate","survey_name"]]  # 选取需要研究的变量
df_1a1 = df_1a1.rename(columns={'trolley_1_rate': 'rate'})  # 统一命名变量，便于汇总数据

# 研究1a第二个材料（footbridge switch）
df_1a2 = df[["trolley_2_rate", "trolley_attention","survey_name"]]  # 选取与材料有关的变量
df_1a2.dropna(axis=0, how='any', inplace=True)  # 删除所有存在缺失值的行
df_1a2 = df_1a2[(df_1a2['trolley_attention'] == 3)]  # 排除注意力测试失败被试的数据
df_1a2 = df_1a2[["trolley_2_rate","survey_name"]]
df_1a2 = df_1a2.rename(columns={'trolley_2_rate': 'rate'})  # 统一命名变量，便于汇总数据

# 研究1b第一个材料（footbridge pole）
df_1b1 = df[["speedboat_1_rate", "speedboat_attention","survey_name"]]  # 选取与材料有关的变量
df_1b1.dropna(axis=0, how='any', inplace=True)  # 删除所有存在缺失值的行
df_1b1 = df_1b1[(df_1b1['speedboat_attention'] == 1)]  # 排除注意力测试失败被试的数据
df_1b1 = df_1b1[["speedboat_1_rate","survey_name"]]  # 选取需要研究的变量
df_1b1 = df_1b1.rename(columns={'speedboat_1_rate': 'rate'})  # 统一命名变量，便于汇总数据

# 研究1b第二个材料（footbridge switch）
df_1b2 = df[["speedboat_2_rate", "speedboat_attention","survey_name"]]  # 选取与材料有关的变量
df_1b2.dropna(axis=0, how='any', inplace=True)  # 删除所有存在缺失值的行
df_1b2 = df_1b2[(df_1b2['speedboat_attention'] == 2)]  # 排除注意力测试失败被试的数据
df_1b2 = df_1b2[["speedboat_2_rate","survey_name"]]  # 选取需要研究的变量
df_1b2 = df_1b2.rename(columns={'speedboat_2_rate': 'rate'})  # 统一命名变量，便于汇总数据

# 研究2a第一个材料（standard switch）
df_2a1 = df[["trolley_3_rate", "trolley_attention","survey_name"]]  # 选取与材料有关的变量
df_2a1.dropna(axis=0, how='any', inplace=True)  # 删除所有存在缺失值的行
df_2a1 = df_2a1[(df_2a1['trolley_attention'] == 4)]  # 排除注意力测试失败被试的数据
df_2a1 = df_2a1[["trolley_3_rate","survey_name"]]  # 选取需要研究的变量
df_2a1 = df_2a1.rename(columns={'trolley_3_rate': 'rate'})  # 统一命名变量，便于汇总数据

# 研究2a第二个材料（standard footbridge）
df_2a2 = df[["trolley_4_rate", "trolley_attention","survey_name"]]  # 选取与材料有关的变量
df_2a2.dropna(axis=0, how='any', inplace=True)  # 删除所有存在缺失值的行
df_2a2 = df_2a2[(df_2a2['trolley_attention'] == 1)]  # 排除注意力测试失败被试的数据
df_2a2 = df_2a2[["trolley_4_rate","survey_name"]]  # 选取需要研究的变量
df_2a2 = df_2a2.rename(columns={'trolley_4_rate': 'rate'})  # 统一命名变量，便于汇总数据

# 研究2a第三个材料（loop）
df_2a3 = df[["trolley_5_rate", "trolley_attention","survey_name"]]  # 选取与材料有关的变量
df_2a3.dropna(axis=0, how='any', inplace=True)  # 删除所有存在缺失值的行
df_2a3 = df_2a3[(df_2a3['trolley_attention'] == 5)]  # 排除注意力测试失败被试的数据
df_2a3 = df_2a3[["trolley_5_rate","survey_name"]]  # 选取需要研究的变量
df_2a3 = df_2a3.rename(columns={'trolley_5_rate': 'rate'})  # 统一命名变量，便于汇总数据

# 研究1b第四个材料（obstacle collide）
df_2a4 = df[["trolley_6_rate", "trolley_attention","survey_name"]]  # 选取与材料有关的变量
df_2a4.dropna(axis=0, how='any', inplace=True)  # 删除所有存在缺失值的行
df_2a4 = df_2a4[(df_2a4['trolley_attention'] == 6)]  # 排除注意力测试失败被试的数据
df_2a4 = df_2a4[["trolley_6_rate","survey_name"]]  # 选取需要研究的变量
df_2a4 = df_2a4.rename(columns={'trolley_6_rate': 'rate'})  # 统一命名变量，便于汇总数据

# 研究2b第一个材料（standard switch）
df_2b1 = df[["speedboat_3_rate", "speedboat_attention","survey_name"]]  # 选取与材料有关的变量
df_2b1.dropna(axis=0, how='any', inplace=True)  # 删除所有存在缺失值的行
df_2b1 = df_2b1[(df_2b1['speedboat_attention'] == 3)]  # 排除注意力测试失败被试的数据
df_2b1 = df_2b1[["speedboat_3_rate","survey_name"]]  # 选取需要研究的变量
df_2b1 = df_2b1.rename(columns={'speedboat_3_rate': 'rate'})  # 统一命名变量，便于汇总数据

# 研究2b第二个材料（standard footbridge）
df_2b2 = df[["speedboat_4_rate", "speedboat_attention","survey_name"]]  # 选取与材料有关的变量
df_2b2.dropna(axis=0, how='any', inplace=True)  # 删除所有存在缺失值的行
df_2b2 = df_2b2[(df_2b2['speedboat_attention'] == 4)]  # 排除注意力测试失败被试的数据
df_2b2 = df_2b2[["speedboat_4_rate","survey_name"]]  # 选取需要研究的变量
df_2b2 = df_2b2.rename(columns={'speedboat_4_rate': 'rate'})  # 统一命名变量，便于汇总数据

# 研究2b第三个材料（loop）
df_2b3 = df[["speedboat_5_rate", "speedboat_attention","survey_name"]]  # 选取与材料有关的变量
df_2b3.dropna(axis=0, how='any', inplace=True)  # 删除所有存在缺失值的行
df_2b3 = df_2b3[(df_2b3['speedboat_attention'] == 2)]  # 排除注意力测试失败被试的数据
df_2b3 = df_2b3[["speedboat_5_rate","survey_name"]]  # 选取需要研究的变量
df_2b3 = df_2b3.rename(columns={'speedboat_5_rate': 'rate'})  # 统一命名变量，便于汇总数据

# 研究2b第四个材料（obstacle collide）
df_2b4 = df[["speedboat_6_rate", "speedboat_attention","survey_name"]]  # 选取与材料有关的变量
df_2b4.dropna(axis=0, how='any', inplace=True)  # 删除所有存在缺失值的行
df_2b4 = df_2b4[(df_2b4['speedboat_attention'] == 5)]  # 排除注意力测试失败被试的数据
df_2b4 = df_2b4[["speedboat_6_rate","survey_name"]]  # 选取需要研究的变量
df_2b4 = df_2b4.rename(columns={'speedboat_6_rate': 'rate'})  # 统一命名变量，便于汇总数据

# 输出各研究最终筛选出的数据数
print("Final sample")
print("Study_1a:", len(df_1a1)+len(df_1a2))
print("Study_1b:", len(df_1b1)+len(df_1b2))
print("Study_2a:", len(df_2a1)+len(df_2a2)+len(df_2a3)+len(df_2a4))
print("Study_2b:", len(df_2b1)+len(df_2b2)+len(df_2b3)+len(df_2b4))


Final sample
Study_1a: 1569
Study_1b: 1426
Study_2a: 3984
Study_2b: 3513



![Image Name](https://cdn.kesci.com/upload/s6bi33dsfn.png?imageView2/0/w/960/h/960)  

* 对比原文献的筛选结果可以发现，我们对预处理数据的筛选结果与原文献完全相同。

### 第一部分（复刻Greene等人的研究1，测试个人力量在道德判断中作用的普遍性）

#### Step1：明确研究问题  
* 本部分复刻了Greene等人的研究1，测试了个人力量在道德判断中作用的普遍性。  
* 在给被试呈现的材料中，a组的材料是标准的道德两难问题，b组的材料是更强调个人力量的道德两难问题。根据被试分别对a组、b组中相同情境的道德评分，我们通过t检验，来研究个人力量在道德判断中作用的普遍性。

#### Step2：汇总需要使用的数据

In [65]:
# 分别计算a组、b组每一个数据的概率密度函数的值并添加到数据框中
# 获取要计算概率密度的列
values1 = df_1a1_w['rate']
values2 = df_1a2_w['rate']
values3 = df_1b1['rate']
values4 = df_1b2['rate']

# 执行核密度估计
kde = gaussian_kde(values1)
kde = gaussian_kde(values2)
kde = gaussian_kde(values3)
kde = gaussian_kde(values4)

# 计算概率密度函数的值
pdf_values1 = kde(values1)
pdf_values2 = kde(values2)
pdf_values3 = kde(values3)
pdf_values4 = kde(values4)

# 将概率密度函数的值添加到数据框中
df_1a1_w['pdf'] = pdf_values1
df_1a2_w['pdf'] = pdf_values2
df_1b1['pdf'] = pdf_values3
df_1b2['pdf'] = pdf_values4

In [91]:
# 设置索引
df_1a1["index"] = range(len(df_1a1))
df_1a1 = df_1a1.set_index("index")
df_1a2["index"] = range(len(df_1a2))
df_1a2 = df_1a2.set_index("index")
df_1b1["index"] = range(len(df_1b1))
df_1b1 = df_1b1.set_index("index")
df_1b2["index"] = range(len(df_1b2))
df_1b2 = df_1b2.set_index("index")

df_1a1_w = df_1a1[df_1a1['survey_name'] == 'PSA006_Western']
df_1a2_w = df_1a2[df_1a2['survey_name'] == 'PSA006_Western']
df_1a1_e = df_1a1[df_1a1['survey_name'] == 'PSA006_Eastern']
df_1a2_e = df_1a2[df_1a2['survey_name'] == 'PSA006_Eastern']
df_1a1_s = df_1a1[df_1a1['survey_name'] == 'PSA006_Southern']
df_1a2_s = df_1a2[df_1a2['survey_name'] == 'PSA006_Southern']

df_1a1_w['force'] = 0
df_1a2_w['force'] = 1
df_1a_w = pd.concat([df_1a1_w, df_1a2_w])

#### Step3：设定模型&选择先验

In [66]:
with pm.Model() as model1:
    # 定义先验分布
    # 我们假设a1组、a2组数据均值的先验分布均为平均值为5，标准差为2.5的正态分布，假设数据标准差为0.4
    mu1 = pm.Normal('mu1', mu=5, sigma=2.5)
    mu2 = pm.Normal('mu2', mu=5, sigma=2.5)
    sigma = 0.4

    # 定义似然
    likelihood1 = pm.Normal(
        "y_est1",
        mu=mu1,
        sigma=sigma,
        observed=df_1a1_w.rate)
    likelihood2 = pm.Normal(
        "y_est2",
        mu=mu2,
        sigma=sigma,
        observed=df_1a2_w.rate)


**🤔为什么假设数据标准差为0.4？**  
* 概率密度函数因变量的最大值不超过1，因此我们首先设置初始标准差值为0.1，并计算对应的概率密度函数。然后，我们通过迭代增加标准差的值，直到找到一个满足所有取值点上概率密度都不超过1的标准差。在每次循环中，我们重新计算当前标准差下的概率密度函数，并判断是否满足条件。如果满足条件，则输出找到的标准差值。

In [67]:
mu = 0
std = 0.1  # 初始标准差值
x = np.linspace(mu - 4*std, mu + 4*std, 1000)
y = norm.pdf(x, mu, std)  # 初始标准差下的概率密度函数

while max(y) > 1:
    std += 0.1  # 调整标准差
    x = np.linspace(mu - 4*std, mu + 4*std, 1000)
    y = norm.pdf(x, mu, std)

print("满足条件的标准差：", std)


满足条件的标准差： 0.4


#### Step4：先验预测检验

In [69]:
# 设置随机种子确保结果可以重复
np.random.seed(84735)

# 根据设定的先验分布，在其中各抽取200个mu1，200个mu2
mu1_200 = np.random.normal(loc=5, scale=2.5, size=200)
mu2_200 = np.random.normal(loc=5, scale=2.5, size=200)

# 将结果存在一个数据框内
prior_pred_sample = pd.DataFrame({"mu1": mu1_200,
                                  "mu2": mu2_200,
                                  "sigma": sigma})
# 查看抽样结果
prior_pred_sample


,mu1,mu2,sigma
0,6.296943,2.153503,0.4
1,4.205120,2.487867,0.4
2,7.289739,2.027098,0.4
3,6.969151,5.994539,0.4
4,4.149370,0.471439,0.4
...,...,...,...
195,1.717442,5.345628,0.4
196,4.888712,7.483474,0.4
197,5.487499,9.981514,0.4
198,6.081147,1.564091,0.4


In [24]:
# 画出每一次的先验预测结果
for i in range(len(prior_pred_sample)):
    mu1 = prior_pred_sample["mu1"][i]
    mu2 = prior_pred_sample["mu2"][i]

    # 绘制概率密度函数图
    x = np.linspace(1, 9, 200)
    y1 = 1 / (np.sqrt(2 * np.pi) * sigma) * \
        np.exp(-(x - mu1)**2 / (2 * sigma**2))
    y2 = 1 / (np.sqrt(2 * np.pi) * sigma) * \
        np.exp(-(x - mu2)**2 / (2 * sigma**2))
    plt.plot(x, y1)
    plt.fill_between(
        x, 0, y1, where=(
            x > norm.ppf(
                0.05, mu1, sigma)) & (
            x < norm.ppf(
                0.95, mu1, sigma)), alpha=0.3)
    plt.plot(x, y2)
    plt.fill_between(
        x, 0, y2, where=(
            x > norm.ppf(
                0.05, mu2, sigma)) & (
            x < norm.ppf(
                0.95, mu2, sigma)), alpha=0.3)
plt.title('prior predictive check')
plt.xlabel('rate')
plt.ylabel('probability density')
plt.legend()
plt.show()


No artists with labels found to put in legend.  Note that artists whose label start with an underscore are ignored when legend() is called with no argument.


<Figure size 640x480 with 1 Axes>

* 研究结果：强有力的证据（Bayes Factor:269.270）表明个人力量对道德判断有影响，这意味着这种影响在文化上是普遍存在的

**我们的先验设置合理吗？**  
* 由图可得，分数（rate）集中在1~9之间，且概率密度函数因变量的最大值不超过1，因此可以认为我们的先验设置比较合理。。。吗？

**🤔为什么我们的先验设置不合理？**  
* 在给被试呈现的材料中，我们要求被试对情境的道德可接受程度进行1~9评分，被试只能打出整数的评分，不能打出非整数的评分（例如3.7），因此分数（rate）是离散型随机变量，应该选择离散型随机变量的分布。正态分布是连续型随机变量的分布，因此我们的先验设置不合理。

#### Step5：重新设定模型&选择先验

In [70]:
with pm.Model() as model2:
    # 定义先验分布
    # 选择多项分布作为先验分布
    prior1 = pm.Dirichlet('prior1', a=np.ones(9))
    prior2 = pm.Dirichlet('prior2', a=np.ones(9))

    # 定义似然
    likelihood1 = pm.Multinomial(
        'likelihood1',
        n=len(df_1a1_w.rate),
        p=prior1,
        observed=np.bincount(df_1a1_w.rate)[
            1:])
    likelihood2 = pm.Multinomial(
        'likelihood2',
        n=len(df_1a2_w.rate),
        p=prior2,
        observed=np.bincount(df_1a2_w.rate)[
            1:])


In [71]:
pm.model_to_graphviz(model2)

#### Step6：先验预测检验

In [72]:
prior_check = pm.sample_prior_predictive(samples=50,
                                   model=model2,
                                   random_seed=84735)


Sampling: [likelihood1, likelihood2, prior1, prior2]


In [73]:
# 对于一次抽样，可以绘制出一条曲线，结合循环绘制出50条曲线
for i in range(prior_check.prior.dims["draw"]):
    sns.lineplot(x=np.arange(1, 10),
                 y=prior_check.prior["prior1"].stack(sample=("chain", "draw"))[:, i], c="grey")
# 设置x、y轴标题和总标题
plt.xlabel("rate",
           fontsize=12)
plt.ylabel("probability density",
           fontsize=12)
plt.suptitle("prior predictive check",
             fontsize=14)
sns.despine()
plt.show()


<Figure size 640x480 with 1 Axes>

**我们的先验设置合理吗？**  
* 由图可得，分数（rate）集中在1~9之间，且概率密度函数因变量的最大值不超过1，因此可以认为我们的先验设置比较合理。

#### Step7：模型评估

In [75]:
# 模型拟合
with model2:
    trace = pm.sample(draws=5000,                   # 使用mcmc方法进行采样，draws为采样次数
                      tune=1000,                    # tune为调整采样策略的次数，可以决定这些结果是否要被保留
                      chains=4,                     # 链数
                      discard_tuned_samples=True,   # tune的结果将在采样结束后被丢弃
                      random_seed=84735)


Auto-assigning NUTS sampler...
Initializing NUTS using jitter+adapt_diag...
Multiprocess sampling (4 chains in 4 jobs)
NUTS: [prior1, prior2]


Sampling 4 chains for 1_000 tune and 5_000 draw iterations (4_000 + 20_000 draws total) took 21 seconds.


In [76]:
# 进行后验预测采样
with model2:
    ppc = pm.sample_posterior_predictive(trace,
                                         random_seed=84735)

Sampling: [likelihood1, likelihood2]


In [77]:
ppc

Inference data with groups:
	> posterior_predictive
	> observed_data

In [32]:
# 提取出所有的后验预测值，使用stack汇总到一起
pre_per_x1 = ppc.posterior_predictive["likelihood1"].stack(
    sample=("chain", "draw"))
pre_per_x2 = ppc.posterior_predictive["likelihood2"].stack(
    sample=("chain", "draw"))

# 提取出所有的观测值
obs_x1 = np.bincount(df_1a1.rate)[1:]
obs_x2 = np.bincount(df_1a2.rate)[1:]

# 计算均值
pre_y1_mean = pre_per_x1.mean(axis=1).values
pre_y2_mean = pre_per_x2.mean(axis=1).values

# 分别提取后验预测均值和对应的观测值，存放在一个表格中
pre_y_mean = pd.DataFrame({
    "posterior_predictive1": pre_y1_mean,
    "observed_data1": obs_x1,
    "posterior_predictive2": pre_y2_mean,
    "observed_data2": obs_x2,
})

# 显示表格
pre_y_mean


,posterior_predictive1,observed_data1,posterior_predictive2,observed_data2
0,224.57360,226,152.12390,153
1,136.50365,137,86.88665,87
2,136.39570,137,110.69815,111
3,82.95470,83,81.94195,82
4,89.92905,90,143.44690,144
5,52.41850,52,101.87435,102
6,41.50590,41,52.40970,52
7,5.91930,5,19.88830,19
8,15.79960,15,33.73010,33


In [33]:
from statistics import median

# 计算预测误差的绝对值
pre_y_mean["pre_error1"] = abs(
    pre_y_mean["observed_data1"] -
    pre_y_mean["posterior_predictive1"])
pre_y_mean["pre_error2"] = abs(
    pre_y_mean["observed_data2"] -
    pre_y_mean["posterior_predictive2"])

# 计算预测误差的中位数
MAE_a = median(pre_y_mean.pre_error1)
MAE_b = median(pre_y_mean.pre_error2)
print(f"MAE_a: {MAE_a:.2f}")
print(f"MAE_b: {MAE_b:.2f}")


MAE_a: 0.51
MAE_b: 0.41


* MAE_a和MAE_b均较小，说明后验模型预测的很准确。

**计算后验预测HDI**

In [34]:
# 定义函数，计算对应X下的观测值超出后验预测模型hdi范围的数量
def count_outlier1(i, hdi=0.94, ppc=ppc):
    # 提取后验预测值
    pre_ys1 = ppc.posterior_predictive["likelihood1"].stack(
        sample=("chain", "draw"))[i].values

    # 提取后验预测值对应的观测值
    true_ys1 = ppc.observed_data["likelihood1"][i].values

    # 计算对应后验预测的hdi的上下限
    lower1, upper1 = az.hdi(pre_ys1, hdi_prob=hdi)

    # 判断是否超过 hdi 边界
    outlier1 = (true_ys1 > upper1) | (true_ys1 < lower1)

    return outlier1

# 定义函数，计算对应X下的观测值超出后验预测模型hdi范围的数量
def count_outlier2(i, hdi=0.94, ppc=ppc):
    # 提取后验预测值
    pre_ys2 = ppc.posterior_predictive["likelihood2"].stack(
        sample=("chain", "draw"))[i].values

    # 提取后验预测值对应的观测值
    true_ys2 = ppc.observed_data["likelihood2"][i].values

    # 计算对应后验预测的hdi的上下限
    lower2, upper2 = az.hdi(pre_ys2, hdi_prob=hdi)

    # 判断是否超过 hdi 边界
    outlier2 = (true_ys2 > upper2) | (true_ys2 < lower2)

    return outlier2


In [35]:
# 建立一个list,存储是否超过后验预测范围的判断结果
hdi_verify1 = []
hdi_verify2 = []

# 使用循环，判断所有X对应的Y是否超出后验预测范围，并计数
for i in range(9):
    hdi_verify1.append(count_outlier1(i, hdi=0.94))
total_number1 = sum(hdi_verify1)
for i in range(9):
    hdi_verify2.append(count_outlier2(i, hdi=0.94))
total_number2 = sum(hdi_verify2)

print(f"a1组所有超过后验预测范围HDI的数量: {total_number1}")
print(f"a2组所有超过后验预测范围HDI的数量: {total_number2}")


a1组所有超过后验预测范围HDI的数量: 0
a2组所有超过后验预测范围HDI的数量: 0


* 这表明，我们的预测准确率接近100%。

#### Step8：统计推断

In [36]:
with pm.Model() as model2:
    # 定义先验分布
    # 选择多项分布作为先验分布
    prior1 = pm.Dirichlet('prior1', a=np.ones(9))
    prior2 = pm.Dirichlet('prior2', a=np.ones(9))

    # 定义似然
    likelihood1 = pm.Multinomial(
        'likelihood1',
        n=len(df_1a1.rate),
        p=prior1,
        observed=np.bincount(df_1a1.rate)[
            1:])
    likelihood2 = pm.Multinomial(
        'likelihood2',
        n=len(df_1a2.rate),
        p=prior2,
        observed=np.bincount(df_1a2.rate)[
            1:])


In [37]:
with model2:
    trace = pm.sample(
        draws=5000,                   # 使用mcmc方法进行采样，draws为采样次数
        tune=1000,                    # tune为调整采样策略的次数，可以决定这些结果是否要被保留
        chains=4,                     # 链数
        discard_tuned_samples=True,  # tune的结果将在采样结束后被丢弃
        random_seed=84735)


Auto-assigning NUTS sampler...
Initializing NUTS using jitter+adapt_diag...
Multiprocess sampling (4 chains in 4 jobs)
NUTS: [prior1, prior2]


Sampling 4 chains for 1_000 tune and 5_000 draw iterations (4_000 + 20_000 draws total) took 20 seconds.


In [86]:
trace.posterior['prior1']

<xarray.DataArray 'prior1' (chain: 4, draw: 5000, prior1_dim_0: 9)>
array([[[0.28736814, 0.21473434, 0.14359285, ..., 0.03815269,
         0.01701372, 0.01063591],
        [0.24222534, 0.1459473 , 0.18068194, ..., 0.06138706,
         0.00704414, 0.01145404],
        [0.29165039, 0.21000539, 0.13475911, ..., 0.03514394,
         0.01527246, 0.01674507],
        ...,
        [0.26058163, 0.20429074, 0.13164856, ..., 0.05211903,
         0.0075398 , 0.02142594],
        [0.2598247 , 0.19530167, 0.14628459, ..., 0.04806566,
         0.0088012 , 0.01858756],
        [0.22468964, 0.17707222, 0.19132672, ..., 0.05456426,
         0.00874748, 0.01673293]],

       [[0.25472304, 0.2199235 , 0.16261102, ..., 0.04562505,
         0.00597535, 0.01115022],
        [0.26362632, 0.14927939, 0.15484473, ..., 0.06381332,
         0.01347013, 0.02389616],
        [0.27994824, 0.19242703, 0.16959027, ..., 0.02972116,
         0.00677252, 0.02429155],
...
        [0.24354741, 0.1387296 , 0.17350921, ..., 0.06125183,
         0.01253322, 0.01838435],
        [0.25416268, 0.13144017, 0.17842683, ..., 0.06245757,
         0.00932043, 0.02589298],
        [0.27503409, 0.21153854, 0.14605912, ..., 0.03255871,
         0.00447189, 0.03523308]],

       [[0.28213912, 0.19735362, 0.18142577, ..., 0.03941898,
         0.00341802, 0.01453548],
        [0.28719517, 0.18445895, 0.15545555, ..., 0.03088969,
         0.01427453, 0.01329977],
        [0.25934354, 0.16375284, 0.17251203, ..., 0.06490953,
         0.00867876, 0.01386413],
        ...,
        [0.27139922, 0.19622631, 0.16494507, ..., 0.03926876,
         0.00896171, 0.01053179],
        [0.26308934, 0.18756302, 0.1723734 , ..., 0.04292309,
         0.00407391, 0.01933249],
        [0.27924386, 0.17293241, 0.14171787, ..., 0.03795689,
         0.00600049, 0.00978019]]])
Coordinates:
  * chain         (chain) int64 0 1 2 3
  * draw          (draw) int64 0 1 2 3 4 5 6 ... 4994 4995 4996 4997 4998 4999
  * prior1_dim_0  (prior1_dim_0) int64 0 1 2 3 4 5 6 7 8

In [87]:
trace.posterior['prior2']

<xarray.DataArray 'prior2' (chain: 4, draw: 5000, prior2_dim_0: 9)>
array([[[0.23295587, 0.08247012, 0.16414737, ..., 0.05793199,
         0.02249775, 0.02113761],
        [0.21159895, 0.0961641 , 0.19042235, ..., 0.05736607,
         0.01540375, 0.02751201],
        [0.22345956, 0.07732806, 0.16480426, ..., 0.03975927,
         0.03041055, 0.03251204],
        ...,
        [0.22387422, 0.0815653 , 0.22710281, ..., 0.03810122,
         0.01487478, 0.02004849],
        [0.21225683, 0.1191352 , 0.19060494, ..., 0.03679861,
         0.01585326, 0.02117429],
        [0.21756463, 0.08701791, 0.13423726, ..., 0.05643111,
         0.02101229, 0.04705774]],

       [[0.19422221, 0.07710359, 0.19143567, ..., 0.05131348,
         0.02479701, 0.02749723],
        [0.24513574, 0.06436846, 0.16838182, ..., 0.05710841,
         0.00821435, 0.02258407],
        [0.20598384, 0.10980287, 0.17605094, ..., 0.05861494,
         0.03870199, 0.02472493],
...
        [0.19403673, 0.09579054, 0.19591453, ..., 0.0340992 ,
         0.02015869, 0.04209185],
        [0.21619391, 0.10412821, 0.17840337, ..., 0.03806623,
         0.02549   , 0.01651165],
        [0.21917688, 0.06718373, 0.15794212, ..., 0.07886765,
         0.02453603, 0.02184032]],

       [[0.18909127, 0.08791213, 0.17330368, ..., 0.05735918,
         0.01651438, 0.02859846],
        [0.2018426 , 0.09406311, 0.17516239, ..., 0.04318972,
         0.01612806, 0.03436253],
        [0.23296054, 0.07731751, 0.16308326, ..., 0.05535948,
         0.01820665, 0.04009605],
        ...,
        [0.2424095 , 0.05872219, 0.18473319, ..., 0.07686282,
         0.00552562, 0.04005141],
        [0.22349309, 0.09701821, 0.15626696, ..., 0.05556894,
         0.01843999, 0.01684501],
        [0.21687949, 0.0991902 , 0.13920304, ..., 0.06184253,
         0.01090954, 0.05892495]]])
Coordinates:
  * chain         (chain) int64 0 1 2 3
  * draw          (draw) int64 0 1 2 3 4 5 6 ... 4994 4995 4996 4997 4998 4999
  * prior2_dim_0  (prior2_dim_0) int64 0 1 2 3 4 5 6 7 8

In [57]:
proportion  = (trace.posterior['prior1'] > trace.posterior['prior2']).mean().values

In [58]:
print(f"虚无假设：a1组评分高于a2组的概率: {proportion:.3f}")
print(f"备择假设：a1组评分低于a2组的概率: {1-proportion:.3f}")

虚无假设：a1组评分高于a2组的概率: 0.457
备择假设：a1组评分低于a2组的概率: 0.543


In [59]:
posterior_odds = (1-proportion)/ proportion
print(f"后验概率比: {posterior_odds:.3f}")

后验概率比: 1.190


In [94]:
prior_dist1 = st.norm(0, 10)
prior_dist2 = st.norm(0, 10)
prior_odds = prior_dist1.cdf(0.2) / prior_dist2.cdf(0.2)
bayes_factor = posterior_odds/prior_odds
print(posterior_odds)
print(prior_odds)
print(f"Bayes Factor(Western):{bayes_factor:.3f}")
print("Bayes Factor(Eestern):1.210")
print("Bayes Factor(Southern):1.390")

1.1896655894052623
1.0
Bayes Factor(Western):1.190
Bayes Factor(Eestern):1.210
Bayes Factor(Southern):1.390
